In [20]:
import csv 
import numpy as np
import random 
import itertools 
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import date
from email import encoders
import os.path

new_entries_file_location = r'C:\\Users\\athomas\\Desktop\\Blind Banter\\new_entries.csv'
previously_matched_pairs_file_location = r'C:\\Users\\athomas\\Desktop\\Blind Banter\\previously_matched_pairs.csv'

# If the number of people registered is an odd number, we inject the dummy evener to make the count even.
dummy_evener = ['Arun Thomas', 'arunthomasaloor@gmail.com'] 

In [10]:
fresh_entries = []

with open(new_entries_file_location) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for entry in csv_reader:
        fresh_entries.append(entry)
csv_file.close()     

fresh_entries.pop(0)

fresh_entries_count = len(fresh_entries)

if fresh_entries_count%2 == 1:
    fresh_entries.append(dummy_evener)
    fresh_entries_count += 1

print('No. of entries in the registration file is',fresh_entries_count)
print('\nThe fresh entries are',fresh_entries)

No. of entries in the registration file is 4

The fresh entries are [['A'], ['B'], ['C'], ['D']]


In [11]:
all_fresh_pairs = list(itertools.combinations(fresh_entries,2))
print('All possible pairs: ',all_fresh_pairs)
print('\nNo. of all possible pairs are',len(all_fresh_pairs))

All possible pairs:  [(['A'], ['B']), (['A'], ['C']), (['A'], ['D']), (['B'], ['C']), (['B'], ['D']), (['C'], ['D'])]

No. of all possible pairs are 6


In [15]:
# Check list of all possible pairs with the output pairs from th previous run. 
# Remove previous output pairs from the possible pairs list.

previous_matched_pairs = []
with open(previously_matched_pairs_file_location, 'r') as filehandle:
    for line in filehandle:
        previous_matched_pairs.append(eval(line.strip()[:-1]))        

allowable_fresh_pairs = [x for x in all_fresh_pairs if x not in previous_matched_pairs]

print('Previously matched pairs: ',previous_matched_pairs)
print('No. of previously matched pairs:',len(previous_matched_pairs))
print('\nCurrently Allowable pairs: ',allowable_fresh_pairs)
print('No. of currently allowable pairs:',len(allowable_fresh_pairs))

Previously matched pairs:  []
No. of previously matched pairs: 0

Currently Allowable pairs:  [(['A'], ['B']), (['A'], ['C']), (['A'], ['D']), (['B'], ['C']), (['B'], ['D']), (['C'], ['D'])]
No. of currently allowable pairs: 6


In [16]:
final_output_pairs  = []
alphaprev = 0
collector = []

for count,item in enumerate(allowable_fresh_pairs):
        alpha = item[0]
        beta = item[1]
        
        if(count == 0): 
            #key value pairs ***
            collector.append(alpha)
            collector.append(beta)
            final_output_pairs.append(item)
            continue    
            
        if alpha in collector or beta in collector:
            continue
            
        final_output_pairs.append(item)
        collector.append(alpha)
        collector.append(beta)
        
for i in final_output_pairs:
    print(i)

(['A'], ['B'])
(['C'], ['D'])


In [17]:
# Writing output pairs to file

with open(previously_matched_pairs_file_location,'a') as fp:    
    fp.write('\n'.join('({},{}),'.format(x[0],x[1]) for x in final_output_pairs))
    fp.write('\n')

<h3>Whatever below is purely related to the mailing mechanism.  </h3>

In [38]:
# Mailing mechanism boilercode only

from datetime import date

friendship_count = 10

final_pairs = [

(['Arun Thomas', 'blindbridge@outlook.com'],['arun aloor', 'arunthomasaloor@gmail.com']),

]

sender_email_address = 'blindbridge@outlook.com'
sender_email_password = 'K!zhak00t'
date = date.today().strftime("%B %d, %Y")

def get_email_subject(name):
    return (f'Dear {name}, Greetings from BlindBridge! Here is your new buddy!')

def get_email_body(receiever, buddy, buddy_email):    
    return '''
Hello {receiever},
            
Your buddy for date is buddy.
You can connect with buddy at buddy_email
            
BlindBridge has helped spark {friendship_count} new friendships so far, and we hope this is the beginning of another amazing friendship!

All pair-ups are completely random, and exclusive to the Duke fraternity.
Please reply to this mail for any feedback or if you want to unsubscribe.

Take care and stay safe,
BlindBridge.            
            '''.format(receiever=receiever,date=date,buddy=buddy,buddy_email =buddy_email,friendship_count = friendship_count)


In [46]:
# Mailing mechanism code

session = smtplib.SMTP('smtp.outlook.com', 587) 
session.starttls() 
session.login(sender_email_address, sender_email_password)

for count,match in enumerate(final_pairs):
    
    email_content_forA = get_email_body(match[0][0], match[1][0], match[1][1])   
    receiver_address_forA = f'{match[0][1]}'    
    message = MIMEMultipart()    
    message['To'] = receiver_address_forA
    message['Subject'] = get_email_subject(match[0][0])
    message.attach(MIMEText(email_content_forA, 'plain'))
    text = message.as_string()
    session.sendmail(sender_email_address, receiver_address, text)
    
    
    email_content_forB = get_email_body(match[1][0], match[0][0], match[0][1])
    receiver_address_forB = f'{match[1][1]}'    
    message = MIMEMultipart()    
    message['To'] = receiver_address_forB
    message['Subject'] = get_email_subject(match[1][0])
    message.attach(MIMEText(email_content_forB, 'plain'))     
    text = message.as_string()
    session.sendmail(sender_email_address, receiver_address, text)
    
    if count == 0: print('Done ↓\n')    
    print(f'{match[0][0]} ←---→ {match[1][0]}')

print('\n')
session.quit()  

Done ↓

Arun Thomas ←---→ arun aloor




(221, b'2.0.0 Service closing transmission channel')